In [1]:
import numpy as np
import pandas as pd
import re
import scipy.stats as st
import sklearn.metrics as met
import matplotlib.pyplot as plt
import sklearn.preprocessing as prep
import time

%matplotlib inline
title = "PPD"
path = "C:/Users/recre/OneDrive/Stat"
icy = 'target'

# PPD
<script>alert('XSS!')</script>

## Initiate Data

Read data from orginal data files, save them into database which is easier to reload

### Master Data
Input:
* Training Master Data:
    * PPD_dat_1.csv: First-round training set of Master data
    * PPD_dat_2.csv: First-round validation set of Master data
    * PPD_dat_3.csv: Second-round training set of Master data
    * PPD_dayt_2_1.csv: First-round public validation set of Y Labels
    * PPD_dayt_2_2.csv: First-round private validation set of Y Labels
* Validation Master Data:
    * PPD_dav.csv: Second-round validation set of Master data
    
Output:
* da: master data
* irt, irv: sample indices of training and validation set

In [2]:
def Read_concat_csv(file, par_csv = {}):
    da = pd.concat(map(lambda x: pd.read_csv(x, **par_csv), file))
    return(da)
def Del_string(xstr):
    xstrc = xstr.strip().strip("市").strip("省")
    if(xstrc == ""):
        xstrc = np.nan
    return(xstrc)

### Historical Records Data
Input:
* LogInfo Data:
    * PPD_daht_1_LogInfo.csv: First-round training set of LogInfo data
    * PPD_daht_2_LogInfo.csv: First-round validation set of LogInfo data
    * PPD_daht_3_LogInfo.csv: Second-round training set of LogInfo data
    * PPD_dahv_LogInfo.csv: Second-round validation set of LogInfo data
* UserupdateInfo Data:
    * PPD_daht_1_Userupdate.csv: First-round training set of UserupdateInfo data
    * PPD_daht_2_Userupdate.csv: First-round validation set of UserupdateInfo data
    * PPD_daht_3_Userupdate.csv: Second-round training set of UserupdateInfo data
    * PPD_dahv_LogInfo.csv: Second-round validation set of UserupdateInfo data
    
Output:
* dah1: historical records data: LogInfo
* dah2: historical records data: UserupdateInfo

In [3]:
def Read_History(file, icid, ictime, par_csv = {}):
    '''Organize Time-Dependent Historical Records
    
    Parameters
    ----------
    file: a list of file name
    icid: column name of id
    ictime: a list of 2 column names: [basetime, recordtime]
    par_csv: other parameters for pd.read_csv
    '''
    par = {"parse_dates": ictime}
    par.update(par_csv)
    dah = Read_concat_csv(file, par)
    dahb = (dah.assign(Id = dah[icid], Time = (dah[ictime[1]] - dah[ictime[0]]).astype('timedelta64[D]')).set_index(["Id", "Time"])
            .drop([icid]+ictime, axis = 1).sort_index())
    return(dahb)